In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.utils import shuffle
import csv

df_all = pd.read_csv('patent_data_2wks.csv')  # use this for running with small data
df_all = df_all[df_all['ABSTRACT'].notnull()]
df_all = df_all[df_all['APPL_ID'].notnull()]

df_all.columns

Index([u'COUNTRY', u'PUB_ID', u'APPL_ID', u'SUMMARY', u'ABSTRACT'], dtype='object')

In [2]:
df_all = df_all.drop_duplicates(subset=[ 'ABSTRACT'], keep=False)

df_all = df_all[:1000] #this will process only forst 1000 records, comment this if you want to run on all 7000 records

In [3]:
def cosine_similarity(vector1, vector2):
    dot_product = sum(p*q for p,q in zip(vector1, vector2))
    magnitude = math.sqrt(sum([val**2 for val in vector1])) * math.sqrt(sum([val**2 for val in vector2]))
    if not magnitude:
        return 0
    return dot_product/magnitude

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text

my_words = ['','()','(),']
my_stop_words = text.ENGLISH_STOP_WORDS.union(my_words)

tokenize = lambda doc: doc.lower().split(" ")
 
sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True,
                                stop_words=set(my_stop_words),
                                max_features = 200, tokenizer=tokenize)
sklearn_representation = sklearn_tfidf.fit_transform(df_all['ABSTRACT'])

#print sklearn_tfidf.get_feature_names()

In [5]:
threshold = 0.9
for index_0, patent_0 in enumerate(sklearn_representation.toarray()):
    for index_1, patent_1 in enumerate(sklearn_representation.toarray()):
        cosine_sim = cosine_similarity(patent_0, patent_1)
        if index_0 <> index_1 and cosine_sim > threshold:
#             print cosine_sim, ',', df_all.iloc[index_0]['APPL_ID'],  ',', df_all.iloc[index_1]['APPL_ID']
            print '----------------------------------------------------'
            print cosine_sim, ',', df_all.iloc[index_0]['APPL_ID'],  ',', df_all.iloc[index_1]['APPL_ID']
            print df_all.iloc[index_0]['ABSTRACT']
            print 
            print df_all.iloc[index_1]['ABSTRACT']
            print '----------------------------------------------------'
            
    if index_0 > 0 and index_0  % 500 == 0:
        print 'Processed records:', index_0
            

----------------------------------------------------
1.0 , 15065288 , 15065304
The present invention is in the field of soybean variety SJ1312299 breeding and development. The present invention particularly relates to the soybean variety SJ1312299 and its seed, cells, germplasm, plant parts, and progeny, and methods of using SJ1312299 in a breeding program.

The present invention is in the field of soybean variety SJ1312462 breeding and development. The present invention particularly relates to the soybean variety SJ1312462 and its seed, cells, germplasm, plant parts, and progeny, and methods of using SJ1312462 in a breeding program.
----------------------------------------------------
----------------------------------------------------
1.0 , 15065304 , 15065288
The present invention is in the field of soybean variety SJ1312462 breeding and development. The present invention particularly relates to the soybean variety SJ1312462 and its seed, cells, germplasm, plant parts, and progeny,